# DATA EXPLORATION

analyzing a dataset of traffic stops in Rhode Island that was collected by the Stanford Open Policing Project.

Before beginning your analysis, it's important that to familiarize with the dataset. 
We will read the dataset into pandas, examine the first few rows, and then count the number of missing values.

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import datetime as dt

In [0]:
# Read 'police.csv' into a DataFrame named ri and set index with date and time combile column 'date' and 'stop_time'
ri = pd.read_csv('PoliceRI2020.csv', parse_dates= [['date', 'stop_time']], index_col='date_stop_time')

# Examine the head of the DataFrame
ri.head()

In [0]:
# Examine the shape of the DataFrame
print(ri.shape)

# Drop the 'county_name' and 'state' columns
ri.drop(['raw_row_number', 'Unnamed: 0', 'contraband_found'], axis='columns', inplace=True)

# rename to meaningful name for driver detail
ri.rename(columns={'subject_race': 'driver_race', 'subject_sex':'driver_gender'}, inplace=True)

# Examine the shape of the DataFrame (again)
print(ri.shape)

In [0]:
ri.info()

# Cleaning and manipulation data

manage the missing value and catergorize to the proper type  
change nan values in contraband columns to True and False

In [0]:
# Count the number of missing values in each column
ri.isnull().sum()

In [0]:
msno.matrix(ri)

In [0]:
# Drop all rows that are missing 'driver_gender'
ri.dropna(subset=['driver_gender'], inplace=True)
print('\n')
# Count the number of missing values in each column (again)
print(ri.isnull().sum())

# Examine the shape of the DataFrame
print(ri.shape)

In [0]:
# check unique value in contraband_ column
ri.contraband_drugs.replace(np.nan, False, inplace= True)
ri.contraband_weapons.replace(np.nan, False, inplace= True)
ri.contraband_alcohol.replace(np.nan, False, inplace= True)

In [0]:
ri.isnull().sum()

In [0]:
msno.matrix(ri)

## check posible value type and how many value in each column

In [0]:
for i in ri.columns:
#     if i != 'raw_row_number' and i != 'date' and i != 'time' and i != 'department_id' and i != 'vehicle_make' and i != 'vehicle_model' and i != 'raw_BasisForStop'and i != 'raw_SearchResultOne' and i != 'raw_SearchResultTwo' and i != 'raw_SearchResultThree':
        print(ri[i].value_counts())

## Using proper dtype

In [0]:
#arrest_made is object then should be change to 'bool'

ri['district'] = ri.district.astype('category')
ri['driver_race'] = ri.driver_race.astype('category')
ri['driver_gender'] = ri.driver_gender.astype('category')
ri['arrest_made'] = ri.arrest_made.astype('bool')
ri['citation_issued'] = ri.citation_issued.astype('bool')
ri['warning_issued'] = ri.warning_issued.astype('bool')
ri['frisk_performed'] = ri.frisk_performed.astype('bool')

ri['reason_for_stop'] = ri.reason_for_stop.astype('category')

print(ri.dtypes)
print('\n')

In [0]:
# change index to timestamp format
ri.index = pd.to_datetime(ri.index)

In [0]:
#check info() for memory usage reduce when group into proper dtype.
print('previously memory usage: 40+ ish MB', '\n')
ri.info()

In [0]:
ri.head()

In [0]:
# check type of index and columns
print(ri.index)
print(ri.columns)

In [0]:
# check for general statistics
ri.describe()

## Exploring the relationship between driver gender and policing

In [0]:
# Count the unique values in 'violation'
print(ri.reason_for_stop.value_counts())
print('\n')

# Express the counts as proportions
print(ri.reason_for_stop.value_counts(normalize= True))

In [0]:
ri.reason_for_stop.value_counts(normalize= True).plot(kind='bar', figsize=(12,5))
plt.tight_layout()
plt.show()

## separate by driver_gender

In [0]:
# slice to new df as only female and male DataFrame
female = ri[ri.driver_gender == 'female']
male = ri[ri.driver_gender == 'male']

# value_counts(normalize= True) on reason_for_stop
print('female reason_for_stop counts \n', female.reason_for_stop.value_counts(normalize= True))
print('\n')
print('male reason_for_stop counts \n', male.reason_for_stop.value_counts(normalize=True))


In [0]:
# Create 2 subplot to compare female and male case for reasone to stop
fig, ax = plt.subplots(1,2, figsize=(18,5), sharey= True)
ax[0].bar(female.reason_for_stop.value_counts(normalize= True).index, female.reason_for_stop.value_counts(normalize= True))
ax[1].bar(male.reason_for_stop.value_counts(normalize= True).index, male.reason_for_stop.value_counts(normalize= True))
ax[0].set_title('female')
ax[1].set_title('male')
plt.setp(ax[0].xaxis.get_majorticklabels(), rotation=90)
plt.setp(ax[1].xaxis.get_majorticklabels(), rotation=90)
plt.tight_layout()

plt.show()

<b><i> the female speeding case is more than male?</i></b>

from 'bar chart show the speeding violate by female more than 65 percent.

In [0]:
# find out how many male and femele for further investigation
print('male on record {} males'.format(male.shape[0]))
print('female on record :',female.shape[0], '\n')
print('=> male and female proportion is different.')

In [0]:
# Create a DataFrame of female drivers stopped for speeding
female_and_speeding = ri[(ri.driver_gender == 'female') & (ri.reason_for_stop == 'Speeding')]

# Create a DataFrame of male drivers stopped for speeding
male_and_speeding = ri[(ri.driver_gender == 'male') & (ri.reason_for_stop == 'Speeding')]

In [0]:
# Compute the stop outcomes for female drivers (as proportions)
print(female_and_speeding.citation_issued.value_counts(normalize= True))

# Compute the stop outcomes for male drivers (as proportions)
print(male_and_speeding.citation_issued.value_counts(normalize= True))

### <i>the data fails to show that gender has an impact on who gets a ticket for speeding.</i>
<i><b>The numbers are similar for males and females: about 95% of stops for speeding result in a ticket. Thus, </i></b>  
<i>So, be careful on assume the comparison on different sample size.

## Does gender affect whose vehicle is searched?

.value_counts() will separate the count for each in pd.series , count() is count every datapoint

In [0]:
# Check the data type of 'search_conducted'
print(ri.search_conducted.dtypes)

# Calculate the search rate by counting the values
print(ri.search_conducted.value_counts(normalize=True))

# Calculate the search rate by taking the mean
print(ri.search_conducted.mean())

the search rate is about 3.8%.

## Is the search rate varies by driver gender.

In [0]:
# Calculate the search rate for both groups simultaneously
print(ri.groupby('driver_gender').search_conducted.mean())
print('\n')

# calculate by mean with out groupby() วิธีที่สองได้ผมเหมือนกัน
# print(ri[ri.driver_gender=='female'].search_conducted.mean())
# print(ri[ri.driver_gender=='male'].search_conducted.mean())
print('search was conducted in male 4.4 percent and in female 1.9 percent')

## compare Search_conduct rate that consequence from reason_for_stop and driver_gender

<b>An all-points bulletin (APB)</b> is a broadcast issued from any American or Canadian law enforcement agency to its personnel, or to other law enforcement agencies. It typically contains information about a wanted suspect who is to be arrested or a person of interest, for whom law enforcement officers are to look. They are usually dangerous or missing persons. As used by American police  

An all-points bulletin can also be known as a BOLO or BOL, which stands for "be on (the) look-out". Such an alert may also be called a lookout or ATL ("attempt to locate").

In [0]:
# Calculate the search rate for each combination of gender and violation
print(ri.groupby(['driver_gender', 'reason_for_stop']).search_conducted.mean())

it hard to compare between gender , re order of groupby

In [0]:
ri.groupby(['reason_for_stop', 'driver_gender']).search_conducted.mean().head(30)

For all types of violations, the search rate in males is higher than females  

that from above the max search rate in female is about 3.8%.
that indicated gender does affect on search rate

In [0]:
ri.groupby(['reason_for_stop', 'driver_gender']).search_conducted.mean().unstack(level=-1)

In [0]:
ri.groupby(['reason_for_stop', 'driver_gender']).search_conducted.mean().unstack(level=-1).plot(kind= 'bar', figsize=(15,3))
# plt.tight_layout()
plt.title('Search conducted after stop for each reason_for_stop compare between driver_gender')
plt.show()

## Does gender affect who is frisked during a search?

In [0]:
# Take the sum of 'frisk'
print(ri.frisk_performed.sum())

## compare frisk by gender

In [0]:
# Create a DataFrame of stops in which a search was conducted
searched = ri[ri.search_conducted == True]

# Calculate the overall frisk rate by taking the mean of 'frisk'
print(searched.frisk_performed.mean())

# Calculate the frisk rate for each gender
print(searched.groupby('driver_gender').frisk_performed.mean())

<i>The frisk rate is higher for males than for females around 10 percent, <br>   though we can't conclude that this difference is caused by the driver's gender.

## Chance of drugs founded with search was conducted

In [0]:
ri.groupby('search_conducted').mean() #.contraband_drugs.mean()

<i><b> each searh has change that relate with drug 26.8 percent.

## Chance of illegal contraband founded with search was conducted separated by driver race

In [0]:
ri.loc[ri['search_conducted']== True].groupby('driver_race').mean()[['contraband_weapons','contraband_alcohol', 'contraband_drugs']] 

In [0]:
ri.loc[ri['search_conducted']== True].groupby('driver_race').mean()[['contraband_weapons','contraband_alcohol', 'contraband_drugs']].plot.bar()
plt.tight_layout()
plt.show()

<i><b>from the chart above show no significant connection for specific driver_race to the contraband illegal subject.

## What violations are caught in each district?

pd.crosstab(df.column1,df.column2) # df.column is pd.series
.loc[]

In [0]:
# group the table and see frequency in all districts and violations
all_area = pd.crosstab(ri['district'], ri['reason_for_stop'])
all_area.head(10)

In [0]:
all_area.plot(kind='bar', figsize= (12,5))
plt.tight_layout()
plt.legend(loc="upper left")
plt.show()

<i>each district must have a different population size of people but we saw the most stop is "Speeding".

In [0]:
# all_area_nrm = (all_area-all_area.min())/(all_area.max()-all_area.min()) == need?
# all_area_stded = (all_area-all_area.mean())/all_area.std() == not reasonable to use Standardization 

# How Driver Race influence police activity?

In [0]:
# show each race count
ri.loc[ri.index.year == 2015].driver_race.value_counts()

In [0]:
# find the stop count for each race in year 2015
race_stop = pd.DataFrame(ri.loc[ri.index.year == 2015].driver_race.value_counts())
race_stop

In [0]:
Ri_pop = pd.read_csv('dataset/RIpopulation.csv')
Ri_pop

In [0]:
# proper change the colomn name and adjust table
Ri_pop2015 = pd.DataFrame(Ri_pop.loc[Ri_pop['date'] ==2015])
Ri_pop2015.drop(columns=['date'], inplace= True)
Ri_pop2015 = Ri_pop2015.transpose()
Ri_pop2015.rename(columns={4: "pop2015"},inplace=True)

In [0]:
Ri_pop2015

## Find out the stop rate on each race

by merging 2 tables and add stop_rate column

In [0]:
race_stop2015 = race_stop.merge(Ri_pop2015, left_on=race_stop.index, right_on=Ri_pop2015.index)
race_stop2015.rename(columns={'key_0': "race", 'driver_race':'stop_number', 'pop2015':'num_people'},inplace=True)
race_stop2015['stop_rate'] = race_stop2015['stop_number'] / race_stop2015['num_people']

In [0]:
race_stop2015

In [0]:
race_stop2015.set_index('race').stop_rate.sort_values(ascending= False).plot(kind= 'bar', figsize= (7,3))
plt.xticks(rotation=45)
plt.yticks([0.1,0.2,0.3])
plt.ylabel('stop_rate')
# plt.gca().axes.get_xaxis().set_visible(False) use to hide xtick and x-axis

plt.show()

<b><i> In Rhod Island stop rate for black people is higher than the rest race base on population proportion.
Black drivers are stopped at a rate 3.2 times higher than white drivers. Hispanic drivers are stopped at a rate 1.6 times higher than white drivers.

## Search rates and frisk rate by each race

In [0]:
# fns2015 = ri.loc[ri.index.year == 2015].groupby('driver_race').mean()[['frisk_performed','search_conducted']]
fns2015 = ri.groupby('driver_race').mean()[['frisk_performed','search_conducted']]
fns2015

In [0]:
fns2015.sort_values(by='frisk_performed', ascending=False).plot.bar(color=['brown', 'green'], figsize= (12,5))
plt.xticks(rotation=35)
plt.tight_layout()
plt.show()

In [0]:
# compare value how many fold base on white race 
fns2015.loc['white']['frisk_performed']
fns2015.loc['white']['search_conducted']
fns2015_twhite = fns2015.copy()
fns2015_twhite['div_by_whith_frisk'] = fns2015_twhite.frisk_performed / (fns2015.loc['white']['frisk_performed'])
fns2015_twhite['div_by_whith_sch'] = fns2015_twhite.frisk_performed / (fns2015.loc['white']['search_conducted'])
fns2015_twhite

Here we see that among drivers who were stopped, <i><b>black drivers were searched at a rate 2.19</i></b> times higher than white drivers, and <i><b>Hispanic drivers were searched at a rate 1.92</i></b> times higher than white drivers. <i><b>Black drivers were frisked at a rate 1.16</i></b> times higher than white drivers were, and <i><b>Hispanic drivers were frisked at a rate 1.02</i></b> times higher than white drivers were.

In [0]:
print("Here we see that among drivers who were stopped, black drivers were searched at a rate {:.2f} times higher than white drivers, and Hispanic drivers were searched at a rate {:.2f} times higher than white drivers. Black drivers were frisked at a rate {:.2f} times higher than white drivers were, and Hispanic drivers were frisked at a rate {:.2f} times higher than white drivers were".format(fns2015_twhite.loc['black']['div_by_whith_frisk'], fns2015_twhite.loc['hispanic']['div_by_whith_frisk'], fns2015_twhite.loc['black']['div_by_whith_sch'], fns2015_twhite.loc['hispanic']['div_by_whith_sch']))